# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [3]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [ ]:
env = UnityEnvironment(file_name="...")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
# Declare agent
import sys

# Add the subfolder to sys.path
sys.path.append('./tennis')

from unityagents import UnityEnvironment
import numpy as np
import importlib
import model
import agent
importlib.reload(agent)
importlib.reload(model)

# env = UnityEnvironment(file_name="Reacher_Windows_x86_64/Reacher.exe")
env = UnityEnvironment(file_name="tennis/Tennis.app")
# env = UnityEnvironment(file_name="reacher/Reacher_Multi.app")
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))


# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)
# declare the agent
config = agent.MultiAgentDDPGConfig(
    num_state = state_size,
    num_action = action_size,
    num_agent = num_agents,
    actor_learning_rate=1e-4,
    critic_learning_rate=1e-4,
    batch_size=256,
    update_every_timestamp=1,
    update_time_each_stamp=2,
    discount_factor=0.99,
    replay_buffer_size=1e6,
    weight_decay=0)

print("declaring agent")
agent = agent.MultiAgentDDPG(config)
print("declaring agent done")

output_folder_path = "tennis/weights/"


In [ ]:

# Function to train the network
from collections import deque
import matplotlib.pyplot as plt

def train(agent, env, num_agents, folder_path, n_episodes=2000, max_t=1000):

    all_rewards = []
    all_avg_rewards = []
    rewards_window = deque(maxlen=100)  # last 100 scores

    actor_losses = []
    critic_losses = []
    for i_episode in range(n_episodes):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0 : num_agents]
        total_reward = np.zeros(num_agents)
        agent.reset()
        curr_total_actor_loss = 0.
        curr_total_critic_loss = 0.
        num_update = 0
        for t in range(max_t):
            # get action from the agent based on the curernt states
            action = agent.act(state)
            # update the env based on the action
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0 : num_agents]
            reward = env_info.rewards[0 : num_agents]
            done = env_info.local_done[0 : num_agents]
            # update the agent
            curr_actor_loss, curr_critic_loss, updated = agent.step(state, action, reward, next_state, done)
            if updated:
                curr_total_actor_loss += curr_actor_loss
                curr_total_critic_loss += curr_critic_loss
                num_update += 1

            # update for the next iteration
            state = next_state
            total_reward += reward
            # the episode reachs the end, so need to start a new episode
            if np.any( done[0 : num_agents] ):
                break
        if 0 != num_update:
            actor_losses.append(curr_total_actor_loss / float(num_update))
            critic_losses.append(curr_total_critic_loss / float(num_update))
        all_rewards.append(np.max(total_reward))
        rewards_window.append(np.max(total_reward))
        all_avg_rewards.append(np.mean(rewards_window))
        print('Episode {}\tAverage Score: {:.2f}\t Total Score: {:.2f}'.format(i_episode, np.mean(rewards_window), np.mean(total_reward)))
        if i_episode % 100 == 0:
            print('Episode {}\tAverage Score: {:.2f}\t Total Score: {:.2f}'.format(i_episode, np.mean(rewards_window), np.mean(total_reward)))
        if np.mean(rewards_window)>=0.5:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(rewards_window)))
            break
        # if i_episode % 200 == 0:
        #     agent.save_model(folder_path, '_ep_' + str(i_episode))

    plt.figure(figsize=(10, 5))
    # plt.plot(actor_losses, label='Actor Loss')
    plt.plot(critic_losses, label='Critic Loss')
    plt.xlabel('Episode')
    plt.ylabel('Critic Loss')
    plt.title('MADDPG Critic Loss Curve')
    plt.legend()
    plt.show()
    plt.figure(figsize=(10, 5))
    plt.plot(actor_losses, label='Actor Loss')
    # plt.plot(critic_losses, label='Critic Loss')
    plt.xlabel('Episode')
    plt.ylabel('Actor Loss')
    plt.title('MADDPG Actor Loss Curve')
    plt.legend()
    plt.show()

    return all_rewards, all_avg_rewards

scores, avg_scores = train(agent, env, num_agents, output_folder_path, 200, 1000)
env.close()

# # save the trained weight
# agent.save_model(output_folder_path)

# plot the score distribution
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores, color = 'blue', label = 'Score Each Episode')
plt.plot(np.arange(len(avg_scores)), avg_scores, color = 'red', label = 'Average Score')
# Optionally, set the y-axis limits to cover both data sets
plt.ylabel('Rewards')
plt.xlabel('Episode #')
plt.show()

In [ ]:
# Test the model
# Declare agent
import sys

# Add the subfolder to sys.path
sys.path.append('./reacher')

from unityagents import UnityEnvironment
import numpy as np
import importlib
import model
import agent
importlib.reload(agent)
importlib.reload(model)

# env = UnityEnvironment(file_name="Reacher_Windows_x86_64/Reacher.exe")
env = UnityEnvironment(file_name="reacher/Reacher_Multi.app")
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))


# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)
# declare the agent
config = agent.DDPGConfig(
    num_state = state_size,
    num_action = action_size,
    num_agent = num_agents,
    actor_learning_rate=1e-4,
    critic_learning_rate=1e-4,
    batch_size=512,
    update_every_timestamp=20,
    update_time_each_stamp=20,
    discount_factor=0.99,
    replay_buffer_size=1e6,
    weight_decay=0)
output_folder_path = "tennis/weights/"
agent = agent.DDPGAgent(config)
agent.load_model(output_folder_path)

env_info = env.reset(train_mode=False)[brain_name] # reset the environment
states = env_info.vector_observations              # get the current state
cnt  = 1
for _ in range(3000):
    print(cnt)
    cnt+=1
    actions = agent.act(states)
    env_info = env.step(actions)[brain_name]
    states = env_info.vector_observations
    if np.any(env_info.local_done):
        break
env.close()

